# Time variability

In this tutorial, we will cover how to instatiate a time-variable `StarryProcess`, useful for modeling stars with spots that evolve over time. We will show how to sample from the process and use it to do basic inference.

In [ ]:
%matplotlib inline

In [ ]:
%config InlineBackend.figure_format = "retina"

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Disable annoying font warnings
matplotlib.font_manager._log.setLevel(50)

# Disable theano deprecation warnings
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=matplotlib.MatplotlibDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="theano")

# Style
plt.style.use("default")
plt.rcParams["savefig.dpi"] = 100
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.figsize"] = (12, 4)
plt.rcParams["font.size"] = 14
plt.rcParams["text.usetex"] = False
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = ["Liberation Sans"]
plt.rcParams["font.cursive"] = ["Liberation Sans"]
try:
    plt.rcParams["mathtext.fallback"] = "cm"
except KeyError:
    plt.rcParams["mathtext.fallback_to_cm"] = True
plt.rcParams["mathtext.fallback_to_cm"] = True

# Short arrays when printing
np.set_printoptions(threshold=0)

In [ ]:
del matplotlib
del plt
del warnings

## Setup

In [ ]:
from starry_process import StarryProcess
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import theano
import theano.tensor as tt

To instantiate a time-variable `StarryProcess`, we simply pass a nonzero value for the `tau` parameter:

In [ ]:
sp = StarryProcess(tau=25.0)

This is the timescale of the surface evolution in arbitrary units (i.e., this will have the same units as the rotation period and the input time arrays; units of days are the common choice). We can also provide a GP kernel to model the time variability. By default a Matern-3/2 kernel is used, but that can be changed by supplying any of the kernels defined in the `starry_process.temporal` module with the `kernel` keyword. If you wish, you can even provide your own callable tensor-valued function of the form

```python
def kernel(t1, t2, tau):
    (...)
    return K
```

where `t1` and `t2` are the input times (scalars or vectors), `tau` is the timescale, and `K` is a covariance matrix of shape ``(len(t1), len(t2))``. 

Let's stick with the `Matern32` kernel for now, and specify a time array over which we'll evaluate the process:

In [ ]:
t = np.linspace(0, 50, 1000)

## Sampling

### Sampling in spherical harmonics

The easiest thing we can do is sample maps. For time-variable processes, we can pass a time `t` argument to `sample_ylm` to get map samples evaluated at different points in time:

In [ ]:
y = sp.sample_ylm(t).eval()
y

Note the shape of `y`, which is `(number of samples, number of times, number of ylms)`:

In [ ]:
y.shape

At every point in time, the spherical harmonic representation of the surface is different. We can visualize this as a movie by simply calling

```python
sp.visualize(y)
```

In [ ]:
# We actually tweak the contrast a little,
# and downsample to make this run quicker
sp.visualize(y[:, ::10], vmin=0.6, vmax=1.3)

Computing the corresponding light curve is easy:

In [ ]:
flux = sp.flux(y, t).eval()
flux

where the shape of `flux` is `(number of samples, number of times)`:

In [ ]:
flux.shape

We could also pass explicit values for the following parameters (otherwise they assume their default values):

In [ ]:
from IPython.display import display, Markdown
from starry_process.defaults import defaults

defaults["u"] = defaults["u"][: defaults["udeg"]]
display(
    Markdown(
        """
| attribute | description | default value |
| - | :- | :-:
| `i` | stellar inclination in degrees | `{i}` |
| `p` | stellar rotation period in days | `{p}`|
| `u` | limb darkening coefficient vector | `{u}` |
""".format(
            **defaults
        )
    )
)

Here's the light curve in parts per thousand:

In [ ]:
plt.plot(t, 1e3 * flux[0])
plt.xlabel("rotations")
plt.ylabel("relative flux [ppt]")
plt.show()

### Sampling in flux

We can also sample in flux directly:

In [ ]:
flux = sp.sample(t, nsamples=50).eval()
flux

where again it's useful to note the shape of the returned quantity, `(number of samples, number of time points)`:

In [ ]:
flux.shape

Here are all 50 light curves plotted on the same scale:

In [ ]:
fig, ax = plt.subplots(10, 5, figsize=(12, 8), sharex=True, sharey=True)
ax = ax.flatten()
for k in range(50):
    ax[k].plot(t, 1e3 * flux[k], lw=0.5)
    ax[k].axis("off")

## Doing inference

We can also do inference using time-variable `StarryProcess` models. Let's do a mock ensemble analysis on the 50 light curves we generated above. First, let's add some observation noise. Here's what the first "observed" light curve looks like:

In [ ]:
ferr = 1e-3
np.random.seed(0)
f = flux + ferr * np.random.randn(50, len(t))
plt.plot(t, flux[0], "C0-", lw=0.75, alpha=0.5)
plt.plot(t, f[0], "C0.", ms=3)
plt.xlabel("time [days]")
plt.ylabel("relative flux [ppt]")
plt.show()

Now, let's try to infer the timescale of the generating process. For simplicity, we'll keep all other parameters fixed at their default (and in this case, true) values. As in the [Quickstart](Quickstart.ipynb) tutorial, we compile the likelihood function using `theano`. It will accept two inputs, a light curve and a timescale, and will return the corresponding log likelihood. To make this example run a little faster, we'll also downsample the light curves by a factor of 5 (not recommended in practice! We should never throw out information!)

In [ ]:
f_tensor = tt.dvector()
tau_tensor = tt.dscalar()
log_likelihood = theano.function(
    [f_tensor, tau_tensor],
    StarryProcess(tau=tau_tensor).log_likelihood(t[::5], f_tensor[::5], ferr ** 2),
)

Compute the joint likelihood of all datasets:

In [ ]:
tau = np.linspace(0, 50, 100)
ll = np.zeros_like(tau)
for k in tqdm(range(len(tau))):
    ll[k] = np.sum([log_likelihood(f[n], tau[k]) for n in range(50)])

Following the same steps as in the [Quickstart](Quickstart.ipynb) tutorial, we can convert this into a posterior distribution by normalizing it (and implicitly assuming a uniform prior over `tau`):

In [ ]:
likelihood = np.exp(ll - np.max(ll))
prob = likelihood / np.trapz(likelihood, tau)
plt.plot(tau, prob, label="posterior")
plt.axvline(25, color="C1", label="truth")
plt.legend()
plt.ylabel("probability density")
plt.xlabel("variability timescale [days]")
plt.show()

As expected, we correctly infer the timescale of variability.